In [2]:
import pandas as pd
from sqlalchemy import create_engine
import xml.etree.ElementTree as ET

In [4]:
import pandas as pd

# Corrigindo a lista de arquivos com vírgulas entre os caminhos
xml_files = [
    r"C:\Users\Felipe\Desktop\Banco de Dados\biblia\xml\aa.min.xml",
    r"C:\Users\Felipe\Desktop\Banco de Dados\biblia\xml\acf.min.xml",
    r"C:\Users\Felipe\Desktop\Banco de Dados\biblia\xml\nvi.min.xml"
]

# Criar um DataFrame combinando todos os arquivos
all_dfs = []

for file in xml_files:
    df = pd.read_xml(file)
    all_dfs.append(df)

# Concatenar todos em um único DataFrame
df_biblia = pd.concat(all_dfs, ignore_index=True)

# Exibir as primeiras linhas
print(df_biblia.head())


           name abbrev  chapters         c
0       Gênesis     gn        50  \n\t\t\t
1         Êxodo     ex        40  \n\t\t\t
2      Levítico     lv        27  \n\t\t\t
3       Números     nm        36  \n\t\t\t
4  Deuteronômio     dt        34  \n\t\t\t


In [ ]:
import mysql.connector

# Conectar ao banco
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Sen3180326",
    database="labstag"
)

cursor = conn.cursor()

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

# Lista com todos os arquivos + nome da versão
xml_files = [
    ("aa", r"C:\Users\Felipe\Desktop\Banco de Dados\biblia\xml\aa.min.xml"),
    ("acf", r"C:\Users\Felipe\Desktop\Banco de Dados\biblia\xml\acf.min.xml"),
    ("nvi", r"C:\Users\Felipe\Desktop\Banco de Dados\biblia\xml\nvi.min.xml")
]

for versao, xml_path in xml_files:
    print(f"Processando versão {versao}...")

    tree = ET.parse(xml_path)
    root = tree.getroot()
    data = []

    for book in root.findall("book"):
        livro = book.get("name")
        abrev = book.get("abbrev")

        for chapter in book.findall("c"):
            capitulo = int(chapter.get("n"))

            for verse in chapter.findall("v"):
                versiculo = int(verse.get("n"))
                texto = verse.text

                data.append((versao, livro, abrev, capitulo, versiculo, texto))

    # Inserir no banco de dados
    insert_query = """
    INSERT INTO versiculos (versao, livro, abrev, capitulo, versiculo, texto)
    VALUES (%s, %s, %s, %s, %s, %s)
    """

    cursor.executemany(insert_query, data)
    conn.commit()
    print(f"Inseridos {len(data)} versículos da versão {versao} com sucesso!")